In [ ]:
from google.colab import drive
drive.mount('/content/drive/') 
# click on the url that pops up and give the necessary authorizations

Mounted at /content/drive/


In [ ]:
cd drive

/content/drive


In [ ]:
cd MyDrive

/content/drive/MyDrive


In [ ]:
cd MGT5399

/content/drive/MyDrive/MGT5399


In [ ]:
DATA_PATH = '/content/drive/MyDrive/MGT5399/'

In [ ]:
PATH_TO_DOWNLOADED = '/content/drive/MyDrive/MGT5399/'

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import os, sys, glob
import gzip
import random
import tqdm
import json
import numpy as np
import pandas as pd
pd.set_option("display.max_columns", None)

from IPython import display
import matplotlib as mpl
from matplotlib import pyplot as plt

In [ ]:
trainingUserDf = pd.read_csv('training-user.csv')
trainingUserDf

,screen_name,isBot
0,faracyaziyor,No
1,elifgibirt,No
2,buocguldurdu,No
3,__baba1yaga_,No
4,hakanyilmazcebi,No
...,...,...
2995,kupakralicesi53,No
2996,muhamme44063672,No
2997,galaopta,Not sure
2998,hygea0,No


In [ ]:
def get_user_info_metadata(user_metadata_line):
    
    user_id = user_metadata_line['id_str']
    user_name = user_metadata_line['name']
    user_screen_name = user_metadata_line['screen_name'].lower()
    user_location = user_metadata_line['location']
    user_description = user_metadata_line['description']
    user_followers_count = user_metadata_line['followers_count']
    user_friends_count = user_metadata_line['friends_count']
    
    dictionary = {'user_id':user_id, 'user_name': user_name, 'user_screen_name':user_screen_name, 'user_location':user_location,
     'user_description':user_description, 'user_followers_count':user_followers_count, 'user_friends_count':user_friends_count}

    return dictionary

In [ ]:
def get_followers_all_ratio(user_followers_count, user_friends_count):
    
    if user_friends_count + user_followers_count == 0:
        followers_all_ratio = 0

    else:
        followers_all_ratio =  user_followers_count / (user_friends_count + user_followers_count)

    return followers_all_ratio

In [ ]:
def get_desc_len(user_description):
    
    description_len = len(user_description)

    return description_len

In [ ]:
dfBot = {'user_id':[],
         'user_name':[],
         'user_screen_name':[],
         'user_location':[],
         'user_description':[],
         'user_followers_count':[],
         'user_friends_count':[],
         'description_len':[],
         'followers_to_all_ratio':[]}

with gzip.open(f"{PATH_TO_DOWNLOADED}user_profiles.jsons.gz", "rb") as f:
    for line in f:
        line = json.loads(line)

        dictionary = get_user_info_metadata(line)
        for k,v in dictionary.items():
            dfBot[k].append(v)

        
        # manually crafted data:
        description_len = get_desc_len(dictionary['user_description'])
        dfBot['description_len'].append(description_len)
        
        followers_all_ratio = get_followers_all_ratio(dictionary['user_followers_count'], 
                                                      dictionary['user_friends_count'])

        dfBot['followers_to_all_ratio'].append(followers_all_ratio)

In [ ]:
def get_retweet_tweet_ratio(line):
    number_retweets = 0
    number_original_tweets = 0

    for tweet in line['tweets']:
        try:
            tweet['retweeted_status']
            number_retweets += 1
                
        except:
            number_original_tweets += 1
            
    total_tweets = number_retweets + number_original_tweets
    
    if total_tweets == 0:
        retweet_total_ratio = None
    else:
        retweet_total_ratio = number_retweets/(total_tweets)
    
    return retweet_total_ratio

In [ ]:
def total_tweets_of_user(line):
    tot_tweet = 0
    for tweet in line['tweets']:
      tot_tweet += 1
    
    return tot_tweet

In [ ]:
def get_median_number_retween_count(line):
    num_median_favorites = np.median([tweet['retweet_count'] for tweet in line['tweets']])

    return num_median_favorites

In [ ]:
def get_median_number_favorites(line):
    num_median_favorites = np.median([tweet['favorite_count'] for tweet in line['tweets']])

    return num_median_favorites

In [ ]:
dfBotTweets = {'user_id':[],
               'retweet_total_ratio':[],
               'num_median_favorites':[],
               'total_tweets':[],
               'is_protected':[],
               'followers_count':[],
               'favourites_count': [],
               'statuses_count':[],
               'friends_count':[],
               'is_verified': [],
               'default_profile_image':[],
               'default_profile':[],
               'num_median_retweets':[],
               'favorites/total':[],
               'following/followers':[]}

i = 0

with gzip.open(f"{PATH_TO_DOWNLOADED}user_tweets.jsons.gz", "rb") as f:
    for line in f:

        line = json.loads(line)
        if line["tweets"] == []:
          dfBotTweets['followers_count'].append(0)
          dfBotTweets['default_profile'].append(0)
          dfBotTweets['default_profile_image'].append(0)
          dfBotTweets['is_protected'].append(0)
          dfBotTweets['is_verified'].append(0)
          dfBotTweets['friends_count'].append(0)
          dfBotTweets['favourites_count'].append(0)
          dfBotTweets['total_tweets'].append(0)
          dfBotTweets['statuses_count'].append(0)
          dfBotTweets['retweet_total_ratio'].append(0)
          dfBotTweets['num_median_favorites'].append(0)
          dfBotTweets['num_median_retweets'].append(0)
          dfBotTweets['favorites/total'].append(0)
          dfBotTweets['following/followers'].append(0)
          user_id = line['user_id']
          dfBotTweets['user_id'].append(user_id)
          continue 



        user_id = line['user_id']
        dfBotTweets['user_id'].append(user_id)

     
        followers_count = line['tweets'][0]['user']['followers_count']
        dfBotTweets['followers_count'].append(followers_count)

        favourites_count = line['tweets'][0]['user']['favourites_count']
        dfBotTweets['favourites_count'].append(favourites_count)

        statuses_count = line['tweets'][0]['user']['statuses_count']
        dfBotTweets['statuses_count'].append(statuses_count)

        friends_count = line['tweets'][0]['user']['friends_count']
        dfBotTweets['friends_count'].append(friends_count)

        is_verified = line['tweets'][0]['user']['verified']
        if is_verified == True:
          dfBotTweets['is_verified'].append(1)
        else:
          dfBotTweets['is_verified'].append(0)
        
        is_protected = line['tweets'][0]['user']['protected']
        if is_protected == True:
          dfBotTweets['is_protected'].append(1)
        else:
          dfBotTweets['is_protected'].append(0)
        
        default_profile_image = line['tweets'][0]['user']['default_profile_image']
        if default_profile_image == True:
          dfBotTweets['default_profile_image'].append(1)
        else:
          dfBotTweets['default_profile_image'].append(0)

        default_profile = line['tweets'][0]['user']['default_profile']
        if default_profile_image == True:
          dfBotTweets['default_profile'].append(1)
        else:
          dfBotTweets['default_profile'].append(0)
        


        
        total_tweets_of_user2 = total_tweets_of_user(line)
        dfBotTweets['total_tweets'].append(total_tweets_of_user2)

        if favourites_count != 0:
          dfBotTweets['favorites/total'].append(total_tweets_of_user2/favourites_count)
        else:
          dfBotTweets['favorites/total'].append(0)
        
        if friends_count != 0:
          dfBotTweets['following/followers'].append(followers_count/friends_count)
        else:
          dfBotTweets['following/followers'].append(0)
        



        retweet_total_ratio = get_retweet_tweet_ratio(line)
        dfBotTweets['retweet_total_ratio'].append(retweet_total_ratio)
        
        num_median_favorites = get_median_number_favorites(line)
        dfBotTweets['num_median_favorites'].append(num_median_favorites)

        num_median_retweets = get_median_number_retween_count(line)
        dfBotTweets['num_median_retweets'].append(num_median_retweets)

        i += 1
        if(i == 4):
          print(line)

        if i % 3000 == 0:
            print(i)

{'tweets': [{'created_at': 'Fri May 13 19:32:18 +0000 2022', 'id': 1525197259482472449, 'id_str': '1525197259482472449', 'text': '#ÖlumcülDeneyAşi https://t.co/2dZ9pj01eo', 'truncated': False, 'entities': {'hashtags': [{'text': 'ÖlumcülDeneyAşi', 'indices': [0, 16]}], 'symbols': [], 'user_mentions': [], 'urls': [{'url': 'https://t.co/2dZ9pj01eo', 'expanded_url': 'https://twitter.com/UyanisMersin/status/1525193463934984194', 'display_url': 'twitter.com/UyanisMersin/s…', 'indices': [17, 40]}]}, 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 1382717948092157953, 'id_str': '1382717948092157953', 'name': 'Murvet', 'screen_name': 'Murvet35838540', 'location': '', 'description': '', 'url': None, 'entities': {'description': {'urls': []}}, 'protected': False, 'followers_c

In [ ]:
dfBotTweets = pd.DataFrame(dfBotTweets)
dfBotTweets

,user_id,retweet_total_ratio,num_median_favorites,total_tweets,is_protected,followers_count,favourites_count,statuses_count,friends_count,is_verified,default_profile_image,default_profile,num_median_retweets,favorites/total,following/followers
0,1044821071,0.090000,57.0,200,0,12263,4476,4297,9275,0,0,0,2.0,0.044683,1.322156
1,1086355147,0.172973,10.0,185,0,14417,233793,2945,9,0,0,0,0.0,0.000791,1601.888889
2,205625453,0.000000,9.0,200,0,729989,43,104196,104,1,0,0,1.0,4.651163,7019.125000
3,1382717948092157953,0.356784,0.0,199,0,89,10574,3026,463,0,0,0,0.0,0.018820,0.192225
4,1302055645307375618,0.646154,0.0,195,0,511,82115,28844,367,0,0,0,7.0,0.002375,1.392371
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29901,1865050056,0.890411,0.0,73,0,12,89,91,26,0,1,1,3.0,0.820225,0.461538
29902,1336329028324577281,0.783920,0.0,199,0,545,2114,4494,625,0,0,0,124.0,0.094134,0.872000
29903,1214219054329675777,0.970588,0.0,170,0,423,10891,32845,945,0,0,0,8.0,0.015609,0.447619
29904,310390437,0.875000,0.0,200,0,30567,200259,72204,31413,0,0,0,375.5,0.000999,0.973068


In [ ]:
dfBot = pd.DataFrame(dfBot)
dfBot

,user_id,user_name,user_screen_name,user_location,user_description,user_followers_count,user_friends_count,description_len,followers_to_all_ratio
0,1195008426784493571,Mesut Demir🎨,grslsntlarogrt,Mardin,Görsel Sanatlar Öğretmeni 🎨,2527,2609,27,0.492017
1,912738858437050369,AdnanToprak,adnantoprak46,Türkiye,,7527,6755,0,0.527027
2,2490178497,"Nejla ÖNÇIRAK😷 bilgin yoksa , fikrin de olmasın👍",oykunejla,izmir,Önce insan...\nAnne❤\nHemşire❤\nHayvansever❤\n...,8819,8591,159,0.506548
3,161774611,Tek Kişilik Muhalefet,yalaaaaannnnn,,Hüloğcular lütfen uzan durun,284,1352,28,0.173594
4,1349294559797370881,Muti_79,muti_79,"Ankara, Türkiye",,3866,3937,0,0.495450
...,...,...,...,...,...,...,...,...,...
31816,467645507,T.C. İçişleri Bakanlığı,tc_icisleri,Türkiye,"T.C. İçişleri Bakanlığı Resmi Twitter Sayfası,...",2408670,29,119,0.999988
31817,1246020601996816385,tuncay,tuncayyilmz,"Gebze, Türkiye",her şeyimi buldum,8191,8183,17,0.500244
31818,601097854,GMG Kastamonuspor Kulübü,gmgkastamonusk,Kastamonu,GMG Kastamonuspor Kulübü Resmi Twitter Hesabı ...,9323,20,82,0.997859
31819,356064164,Show TV,showtv,"İstanbul, Türkiye",Show TV Resmi Twitter Profili https://t.co/d0g...,1355456,111,101,0.999918


In [ ]:
from sklearn.preprocessing import minmax_scale

dfBotTweets[['num_median_favorites','followers_count','favourites_count','statuses_count','friends_count']] = minmax_scale(dfBotTweets[['num_median_favorites','followers_count','favourites_count','statuses_count','friends_count']])

In [ ]:
dfBotAll = dfBot.merge(dfBotTweets,
                       how='left')
dfBotAll[['retweet_total_ratio', 'num_median_favorites']] = dfBotAll[['retweet_total_ratio', 'num_median_favorites']].fillna(0)
dfBotAll

,user_id,user_name,user_screen_name,user_location,user_description,user_followers_count,user_friends_count,description_len,followers_to_all_ratio,retweet_total_ratio,num_median_favorites,total_tweets,is_protected,followers_count,favourites_count,statuses_count,friends_count,is_verified,default_profile_image,default_profile,num_median_retweets,favorites/total,following/followers
0,1195008426784493571,Mesut Demir🎨,grslsntlarogrt,Mardin,Görsel Sanatlar Öğretmeni 🎨,2527,2609,27,0.492017,0.060,0.000055,200.0,0.0,0.000134,0.005427,0.000218,0.002293,0.0,0.0,0.0,0.0,0.022414,0.967483
1,912738858437050369,AdnanToprak,adnantoprak46,Türkiye,,7527,6755,0,0.527027,0.200,0.005908,200.0,0.0,0.000397,0.199768,0.000035,0.005920,0.0,0.0,0.0,9.0,0.000609,1.114091
2,2490178497,"Nejla ÖNÇIRAK😷 bilgin yoksa , fikrin de olmasın👍",oykunejla,izmir,Önce insan...\nAnne❤\nHemşire❤\nHayvansever❤\n...,8819,8591,159,0.506548,0.230,0.000055,200.0,0.0,0.000466,0.044121,0.000572,0.007536,0.0,0.0,0.0,0.0,0.002757,1.026539
3,161774611,Tek Kişilik Muhalefet,yalaaaaannnnn,,Hüloğcular lütfen uzan durun,284,1352,28,0.173594,0.000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1349294559797370881,Muti_79,muti_79,"Ankara, Türkiye",,3866,3937,0,0.495450,1.000,0.000000,200.0,0.0,0.000204,0.105598,0.001493,0.003453,0.0,0.0,0.0,99.5,0.001152,0.981712
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31816,467645507,T.C. İçişleri Bakanlığı,tc_icisleri,Türkiye,"T.C. İçişleri Bakanlığı Resmi Twitter Sayfası,...",2408670,29,119,0.999988,0.735,0.000000,200.0,0.0,0.127340,0.000099,0.000167,0.000025,1.0,0.0,0.0,501.0,1.234568,83084.310345
31817,1246020601996816385,tuncay,tuncayyilmz,"Gebze, Türkiye",her şeyimi buldum,8191,8183,17,0.500244,0.020,0.000055,200.0,0.0,0.000432,0.015331,0.000148,0.007165,0.0,0.0,0.0,0.0,0.007935,1.000979
31818,601097854,GMG Kastamonuspor Kulübü,gmgkastamonusk,Kastamonu,GMG Kastamonuspor Kulübü Resmi Twitter Hesabı ...,9323,20,82,0.997859,0.000,0.001325,200.0,0.0,0.000493,0.000053,0.000218,0.000018,0.0,0.0,0.0,1.0,2.298851,466.150000
31819,356064164,Show TV,showtv,"İstanbul, Türkiye",Show TV Resmi Twitter Profili https://t.co/d0g...,1355456,111,101,0.999918,0.000,0.004224,200.0,0.0,0.071640,0.002291,0.001998,0.000097,1.0,0.0,0.0,11.0,0.053107,12211.981982


In [ ]:
dfBotAll.user_screen_name = dfBotAll.user_screen_name.str.lower()

In [ ]:
dfBotAll_train = dfBotAll.merge(trainingUserDf,
                               left_on='user_screen_name',
                               right_on='screen_name')

dfBotAll_train

,user_id,user_name,user_screen_name,user_location,user_description,user_followers_count,user_friends_count,description_len,followers_to_all_ratio,retweet_total_ratio,num_median_favorites,total_tweets,is_protected,followers_count,favourites_count,statuses_count,friends_count,is_verified,default_profile_image,default_profile,num_median_retweets,favorites/total,following/followers,screen_name,isBot
0,1301168604667355142,Doğabilim,dogabilimorg,,Gerçek Bilginin Işığında!\nYayınlarımıza ulaşm...,803,93,109,0.896205,0.030000,0.000055,200.0,0.0,0.000042,0.000008,0.000018,0.000082,0.0,0.0,0.0,0.0,15.384615,8.634409,dogabilimorg,No
1,628886610,Esra Nur ÖZDEN,esranurmuratozd,Turkey,📍Güzeller güzeli 4 melek annesi\n📍Ak Parti Ank...,1962,1650,134,0.543189,0.985000,0.000000,200.0,0.0,0.000104,0.005485,0.000150,0.001447,0.0,0.0,0.0,165.5,0.022178,1.187879,esranurmuratozd,No
2,547532939,Kübra Şakır,kubussskr,iskenderun,'Allah'tan başka uğruna koştuğun hiçbir şey dü...,531,137,86,0.794910,0.523560,0.000000,191.0,0.0,0.000028,0.007186,0.000156,0.000120,0.0,0.0,0.0,12.0,0.016167,3.897810,kubussskr,No
3,946826934,Gökhan İZGİ 🇹🇷🇦🇿,gokhanizgi34,"İstanbul, Türkiye",📰 Gazeteci - Yazar | 📘 S.O.S.YAL MEDYA kitabı...,42188,33979,146,0.553888,0.320000,0.000055,200.0,0.0,0.002229,0.003431,0.000038,0.029801,0.0,0.0,0.0,0.0,0.035455,1.241531,gokhanizgi34,No
4,802201284833067009,Oral Orhon,orhonoral1975,"İstanbul, Türkiye","Holigan değil, sağlıklı ve Atatürkçü bir BEŞİK...",3988,4995,78,0.443950,0.393939,0.000000,198.0,0.0,0.000211,0.047920,0.002330,0.004382,0.0,0.0,0.0,0.0,0.002513,0.798439,orhonoral1975,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2994,1338444706204835840,laito proof,vtaeskit,eu ar55 childe main,namjoon yalvaririm telefonlarimi ac,270,673,35,0.286320,0.050000,0.000055,200.0,0.0,0.000014,0.042865,0.000542,0.000589,0.0,0.0,0.0,0.0,0.002838,0.399404,vtaeskit,No
2995,323513738,Ezgobar,shaammone,İzmir,humanoid 🏉 🏳️‍🌈,521,330,15,0.612221,0.025000,0.000055,200.0,0.0,0.000028,0.021978,0.000445,0.000290,0.0,0.0,0.0,0.0,0.005535,1.577039,shaammone,No
2996,1451270540669378572,Havva Sert,havvasert19,,,155,199,0,0.437853,0.835052,0.000000,194.0,0.0,0.000008,0.001285,0.000060,0.000175,0.0,0.0,0.0,15.0,0.091813,0.770000,havvasert19,No
2997,1354034188815654912,airi,airilynx,♡🧷she/her 18,#mitski ; i start the day high and it ends so ...,712,585,82,0.548959,0.120000,0.000055,200.0,0.0,0.000038,0.028812,0.000127,0.000514,0.0,0.0,0.0,0.0,0.004222,1.216724,airilynx,No


In [ ]:
dfBotAll_train.fillna(0,inplace = True)
dfBotAll_train.replace([np.inf, -np.inf], 0, inplace=True)

In [ ]:
X = dfBotAll_train[['description_len', 'followers_to_all_ratio', 'retweet_total_ratio', 'num_median_favorites','user_followers_count','user_friends_count','num_median_favorites','followers_count','favourites_count','statuses_count','friends_count','following/followers','favorites/total']]
y = dfBotAll_train.isBot.apply(lambda x: 1 if x=='Yes' else 0)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.20, random_state=42)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error
# create an instance
dtc_bot = DecisionTreeClassifier(min_samples_leaf=50, random_state = 42,criterion = "entropy")

# fit your model
dtc_bot.fit(X_train, y_train)

# make predictions
preds = dtc_bot.predict(X_valid)

# evaluate on validation set
acc_score = accuracy_score(y_valid, preds)
confusion = confusion_matrix(y_valid, preds)
mse = mean_squared_error(y_valid, preds)

print("MSE:", mse, "\n",
      "Accuracy Score:", acc_score, "\n",
      "Confusion Matrix:", "\n", confusion)

MSE: 0.17833333333333334 
 Accuracy Score: 0.8216666666666667 
 Confusion Matrix: 
 [[486  19]
 [ 88   7]]
